In [1]:
#!pip install -q cirq

In [2]:
import cirq
import numpy as np
import sympy

In [3]:
I = cirq.unitary(cirq.I)
sig_x = cirq.unitary(cirq.X)
sig_y = cirq.unitary(cirq.Y)
sig_z = cirq.unitary(cirq.Z)
print(I)
print(sig_x)
print(sig_y)
print(sig_z)

[[1. 0.]
 [0. 1.]]
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


In [4]:
i, x, y, z  = np.random.randint(0,10), np.random.randint(0,10), np.random.randint(0,10), np.random.randint(0,10)
H =  i * I + x * sig_x + y * sig_y + z * sig_z
print(i, x, y, z)
print(H)

4 6 3 1
[[5.+0.j 6.-3.j]
 [6.+3.j 3.+0.j]]


In [5]:
eig_vals, eig_vects = np.linalg.eig(H)
print(eig_vals)
print(eig_vects)

[10.78232998+1.64713361e-16j -2.78232998-1.64713361e-16j]
[[ 0.75744371+0.j         -0.58397193+0.29198597j]
 [ 0.58397193+0.29198597j  0.75744371+0.j        ]]


In [6]:
print(H.dot(eig_vects[:,0]))
print(eig_vals[0]* eig_vects[:,0])
#assert np.all(H.dot(eig_vects[:,0]) == eig_vals[0]* eig_vects[:,0]), "Impossible ?"
#assert np.all(H.dot(eig_vects[:,1]) == eig_vals[1]* eig_vects[:,1]), "Impossible ?"

[8.16700806+2.22044605e-16j 6.29657807+3.14828904e+00j]
[8.16700806+1.24761100e-16j 6.29657807+3.14828904e+00j]


In [7]:
def get_ansatz(ansatz_circ, params):
    resolver = cirq.ParamResolver({rx: params[0], ry: params[1]})
    output_state_vector = cirq.Simulator().simulate(ansatz_circ,resolver)
    return output_state_vector.final_state

In [8]:
def expected_val(ansatz_param):
    q0 = cirq.LineQubit(1)
    ansatz_circ = cirq.Circuit(
        cirq.rx(rx).on(q0),
        cirq.ry(ry).on(q0))

    xm = cirq.X(q0).expectation_from_wavefunction(get_ansatz(ansatz_circ, ansatz_param), {q0: 0})
    ym = cirq.Y(q0).expectation_from_wavefunction(get_ansatz(ansatz_circ, ansatz_param), {q0: 0})
    zm = cirq.Z(q0).expectation_from_wavefunction(get_ansatz(ansatz_circ, ansatz_param), {q0: 0})
    currH = i + x * xm + y * ym + z * zm
    #print(xm, ym, zm, currH)
    return currH.real

In [9]:
from scipy.optimize import minimize

In [10]:
parameters_array = np.array([np.pi, np.pi])
result = minimize(expected_val, parameters_array, method="Powell", tol=1e-3)
print("Matrix Eigen Values: {} and minimum is at: {}".format(eig_vals, min(eig_vals)))
print('The ground state energy: {}'.format(result.fun))

NameError: name 'rx' is not defined